# 誤差逆伝播法でXOR演算が出来るモデルを作成する

インターシップで機械学習の勉強をしに来ている方に、社長が「誤差逆伝播法でXOR演算が出来るモデルを作成する」という課題を出していたので、自分も勉強しつつ、作例を作ってみた。

## 参考にした内容
* ゼロから作るDeep Learning――Pythonで学ぶディープラーニングの理論と実装
https://www.oreilly.co.jp/books/9784873117584/
誤差逆伝播法までのイメージをつかむのと具体的な実装方法

* こちらのGithubのリポジトリ
https://github.com/levelfour/machine-learning-2014/wiki/%E7%AC%AC3%E5%9B%9E---%E5%A4%9A%E5%B1%A4%E3%83%91%E3%83%BC%E3%82%BB%E3%83%97%E3%83%88%E3%83%AD%E3%83%B3
これ見てXORを次元を増やして線形分離をするイメージがつかめた

## 勉強が必要だった事
* パーセプトロンについて
* パーセプトロンでAND,NAND,ORゲート
* 線形分離について
* 活性化関数について
* Loss関数について
* 勾配について
* 誤差逆伝播法について

## 今回書く事
基本的なイメージの理解は飛ばして、実装して確認できるところを書く。
* 誤差逆伝播法でAND演算ができるモデルを作成
実装を通して基本的な学習の流れをつかむ

* 誤差逆伝播法でXOR演算ができるモデルを作成
ANDの場合と同じ構造では、学習が進まない事を確認する
その後、構造を変更して、学習が進むようにする

In [24]:
import numpy as np
import pandas as pd

## 回答データ
ANDの回答データを用意する

In [34]:
and_data = pd.DataFrame(index=['x1', 'x2', 'x3', 'x4'])
and_data

""
x1
x2
x3
x4


## 二乗和誤差
Loss関数として利用する

In [6]:
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)
mean_squared_error(np.random.rand(5), np.random.rand(5))

0.35585081724883094

## シグモイド関数
活性化関数として利用する
STEP関数の形状に近づけるためにbetaを大きくする

In [9]:
def sigmoid(x):
    beta = 10
    return 1 / (1 + np.exp(-x * beta))
sigmoid(0.1)

0.7310585786300049

# 勾配を計算する

In [18]:
def numerical_gradient(f, x):
    print(x)
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x)
        x[idx] = tmp_val - h
        fxh2 = f(x)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
    return grad

# 適当なLOSS関数
def example_function(x):
    return np.sum(x**2)
    
numerical_gradient(example_function, np.array([3.0, 4.0]))

[ 3.  4.]


array([ 6.,  8.])

In [6]:
class SimpleNet:
    def __init__(self):
        self.params = {}
        self.params['W'] = np.random.rand(2)
        self.params['b'] = np.random.rand(1) * -1
        
    def predict(self, x):
        a = np.dot(x, self.params['W']) + self.params['b']
        return sigmoid(a)
        
    def loss(self, x, t):
        y = self.predict(x)
        # 二乗和誤差を利用しているが、
        # クロスエントロピー誤差にしたらもっと学習効率あがるのかもしれない
        return mean_squared_error(y, t)
        
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x,t)
        grads = {}
        grads['W'] = numerical_gradient(loss_W, self.params['W'])
        grads['b'] = numerical_gradient(loss_W, self.params['b'])
        return grads

In [7]:
# 全部で4パターン
x = [[0,0], [0,1], [1,0], [1,1]]

# ANDの回答データ
t = [0,0,0,1]

net = SimpleNet()
net.predict(x)

net.numerical_gradient(x, t)

# 学習率
learning_rate = 0.05

# 誤差がどれだけか記録をつける
train_loss_list = []

# 1万回学習させてみる
for i in range(10000):
    grad = net.numerical_gradient(x, t)
    for key in ('W', 'b'):
        net.params[key] -= learning_rate * grad[key]
        loss = net.loss(x, t)
        train_loss_list.append(loss)

In [8]:
# やればやるほど制度が上がっている事がわかる
train_loss_list

[0.68480903972490215,
 0.58908066864656838,
 0.54198386240785368,
 0.50891591345278475,
 0.50331943784680977,
 0.49522812491146939,
 0.49239193469032477,
 0.48695011453147252,
 0.48427426794479167,
 0.4791246128815948,
 0.47558326674934615,
 0.46954836564195851,
 0.46389613241676531,
 0.45541885934094944,
 0.44500719697069696,
 0.43066219157619517,
 0.4082409986975431,
 0.37847247631620651,
 0.32249301059111485,
 0.25304893536173306,
 0.14791265047059815,
 0.068109256645319027,
 0.042518776407631492,
 0.029425012364105475,
 0.026862636993209385,
 0.025884905236443295,
 0.02407875448433254,
 0.023823369807083904,
 0.022232037288790644,
 0.02209177859820928,
 0.02069043226783384,
 0.020584818395024577,
 0.019349120521778878,
 0.019260694205468564,
 0.018166024864300308,
 0.018089110681858596,
 0.01711420820381359,
 0.01704614657622084,
 0.016173395739495826,
 0.016112553164092873,
 0.015327463242174639,
 0.015272669855566225,
 0.014563284037952906,
 0.014513635878837668,
 0.0138700282821

In [9]:
# 0,0で確認 => Falseのはず
net.predict([0,0])

array([  4.35083650e-07])

In [10]:
# 0,1で確認 => Falseのはず
net.predict([0,1])

array([ 0.00710855])

In [11]:
# 1,0で確認 => Falseのはず
net.predict([1,0])

array([ 0.00710855])

In [12]:
# 1,1で確認 => Trueのはず
net.predict([1,1])

array([ 0.99158326])

# その他触れれて無いこと
* 初期値の与え方
* ミニバッチ学習（4件しかないので、この問題に関しては不要でした）